In [ ]:
!pip install faiss-cpu

In [ ]:
import os
import pandas as pd
import faiss
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm

# **Prepare Dataset**

## Download

In [ ]:
# !gdown ...

In [ ]:
# sudo apt-get install unzip
# !unzip Dataset.zip

## Processing

In [ ]:
dataset_path = 'Dataset'

In [ ]:
os.listdir(dataset_path)

In [ ]:
image_paths = []
labels = []

In [ ]:
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.JPG', '.png', '.jpeg')):
        image_paths.append(os.path.join(dataset_path, filename))
        file_name = filename.split('.')[0]
        label = file_name[7:]
        labels.append(label)

In [ ]:
image_paths

In [ ]:
labels

In [ ]:
df = pd.DataFrame({'image_path': image_paths, 'label': labels})
df

# Store Image like vector

## Vectorize Image

In [ ]:
IMAGE_SIZE = 300
VECTOR_DIM = 300 * 300 * 3  # For RGB images (300x300x3)

In [ ]:
index = faiss.IndexFlatL2(VECTOR_DIM)
label_map = []

In [ ]:
def image_to_vector(image_path):
    """Convert image to normalized vector"""
    # img = Image.open(image_path).resize((IMAGE_SIZE, IMAGE_SIZE))
    img = Image.open(image_path)
    if img.mode == 'RGBA':
        print("Image is RGBA, converting to RGB...")
        # Convert the image to RGB
        img = img.convert('RGB')
    img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
    img_array = np.array(img)
    print("img_array shape", img_array.shape)

    # Handle grayscale images (convert to RGB)
    if len(img_array.shape) == 2:
        img_array = np.stack((img_array,)*3, axis=-1)

    # Normalize pixel values to [0, 1]
    vector = img_array.astype('float32') / 255.0
    return vector.flatten()

In [ ]:
for idx, row in df.iterrows():
    image_path = row['image_path']
    label = row['label']

    try:
        vector = image_to_vector(image_path)
        # Add to Faiss index
        index.add(np.array([vector]))
        label_map.append(label)
    except Exception as e:
        print("image_path: ",image_path)
        print(f"Error processing {image_path}: {e}")

In [ ]:
# Save the index and label map for later use
faiss.write_index(index, "employee_images.index")
np.save("label_map.npy", np.array(label_map))

## Search

In [ ]:
def search_similar_images(query_image_path, k=5):
    """Search for similar employee images"""
    # Load index and labels
    index = faiss.read_index("employee_images.index")
    label_map = np.load("label_map.npy")

    # Convert query image to vector
    query_vector = image_to_vector(query_image_path)

    # Search in Faiss
    distances, indices = index.search(np.array([query_vector]), k)

    # Get results
    results = []
    for i in range(len(indices[0])):
        employee_name = label_map[indices[0][i]]
        distance = distances[0][i]
        results.append((employee_name, distance))

    return results

In [ ]:
def display_query_and_top_matches(query_image_path):
    query_img = Image.open(query_image_path)
    query_img = query_img.resize((300, 300))

    plt.figure(figsize=(5, 5))
    plt.imshow(query_img)
    plt.title("Query Image")
    plt.axis('off')
    plt.show()

    matches = search_similar_images(query_image_path)

    """Display the top 5 matching employee images with distances"""
    # Get the image paths for the results
    top_matches = []
    for name, distance in matches:
        # Find the image path for this employee in df
        img_path = df[df['label'] == name]['image_path'].values[0]
        top_matches.append((name, distance, img_path))
    top_matches

    # Create plot
    plt.figure(figsize=(15, 5))
    for i, (name, distance, img_path) in enumerate(top_matches):
        img = Image.open(img_path)
        img = img.resize((300, 300))

        plt.subplot(1, 5, i+1)
        plt.imshow(img)
        plt.title(f"{name}\nDist: {distance:.2f}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

### Example 1

In [ ]:
# Test Image
# query_image_path1 = "/content/Dataset/Avatar_Amy_Pascal.jpg"
query_image_path1 = "Dataset/Avatar_Amy_Pascal.jpg"

In [ ]:
display_query_and_top_matches(query_image_path1)

### Example 2 (New query image)

In [ ]:
# Stored Image
# query_image_path2 = "/content/test1.jpg"
query_image_path2 = "test1.jpg"

In [ ]:
display_query_and_top_matches(query_image_path2)

### Example 3 (New query image)

In [ ]:
# Stored Image
query_image_path3 = "test2.png"

In [ ]:
display_query_and_top_matches(query_image_path3)

# Store feature of Image

## Vectorize Image

In [ ]:
# Initialize ResNet50 model for feature extraction
model = models.resnet50(pretrained=True)
# Remove the last classification layer
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.eval()

In [ ]:
!pip install facenet-pytorch

In [ ]:
# Install Pretrained model (for face recognition task)

from facenet_pytorch import InceptionResnetV1
face_recognition_model = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((300, 300)), # Resize to the model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalize
])

In [ ]:
def extract_feature(image_path, model):
    """Extract features from an image using a given model."""
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        features = model(img_tensor)
    return features.squeeze().numpy()

In [ ]:
VECTOR_DIM = 512
index = faiss.IndexFlatIP(VECTOR_DIM)
label_map = []

In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    features = extract_feature(row['image_path'], face_recognition_model)
    index.add(np.array([features]))
    label_map.append(row['label'])

In [ ]:
# Save index and labels
faiss.write_index(index, "facenet_features.index")
np.save("facenet_label_map.npy", np.array(label_map))

## Search

In [ ]:
def image_to_feature(image_path, model):
    """Convert image to face embedding using a pre-trained model"""
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0) # Add batch dimension
    # Get the embedding
    with torch.no_grad(): # Disable gradient calculation
        embedding = model(img_tensor)

    # Return the embedding as a numpy array
    return embedding.squeeze().numpy()

In [ ]:
def search_similar_images(query_image_path, k=5):
    """Search for similar employee images using VGG16 features"""
    # Load index and labels
    index = faiss.read_index("facenet_features.index")
    label_map = np.load("facenet_label_map.npy")

    # Convert query image to vector
    query_vector = image_to_feature(query_image_path, face_recognition_model)

    # Search in Faiss
    similarities, indices = index.search(np.array([query_vector]), k)

    # Get results
    results = []
    for i in range(len(indices[0])):
        employee_name = label_map[indices[0][i]]
        similarity = similarities[0][i]
        results.append((employee_name, similarity))

    return results

In [ ]:
def display_query_and_top_matches(query_image_path):
    # Display query image
    query_img = Image.open(query_image_path)
    query_img = query_img.resize((300, 300))
    plt.figure(figsize=(5, 5))
    plt.imshow(query_img)
    plt.title("Query Image")
    plt.axis('off')
    plt.show()

    # Get matches
    matches = search_similar_images(query_image_path)

    # Display top matches
    plt.figure(figsize=(15, 5))
    for i, (name, similarity) in enumerate(matches):
        # Find the image path for this employee
        img_path = df[df['label'] == name]['image_path'].values[0]
        img = Image.open(img_path)
        img = img.resize((300, 300))

        plt.subplot(1, 5, i+1)
        plt.imshow(img)
        plt.title(f"{name}\nSimilarity: {similarity:.2f}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

### Example 1

In [ ]:
# Test Image
query_image_path1 = "Dataset/Avatar_Aaron_Eckhart.jpg"

In [ ]:
display_query_and_top_matches(query_image_path1)

### Example 2 (New query image)

In [ ]:
# Stored Image
query_image_path2 = "test2.png"

In [ ]:
display_query_and_top_matches(query_image_path2)